In [96]:
# Importing dependencies
import pandas as pd
import numpy as np
import os

# Finding Files
files = os.listdir('Resources')

In [97]:
# Change this number to call different files
f = files[1]
df = pd.read_json(os.path.join('Resources',f))
df.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,20,Male,93,Apocalyptic Battlescythe,4.49,Iloni35
1,21,Male,12,Dawne,3.36,Aidaira26
2,17,Male,5,Putrid Fan,2.63,Irim47
3,17,Male,123,Twilight's Carver,2.55,Irith83
4,22,Male,154,Feral Katana,4.11,Philodil43


# Player Count

In [98]:
player_count = len(df.SN.unique())
player_count_df = pd.DataFrame(data=[player_count],columns=['Total Number of Players'])
player_count_df

,Total Number of Players
0,74


# Purchase Analysis (Total)

In [99]:
# Number of unique Items
items = df['Item Name'].unique()
num_unique_items = len(items)

# Average Purchase price
avg_price = df['Price'].mean()

# Total Number of Purchases
total_purchases = len(df['Price'])

# Total Revenue
total_revenue = df['Price'].sum()

# Compiling Analysis into a single output table
pa_df = pd.DataFrame(data=[num_unique_items, avg_price, total_purchases, total_revenue])
pa_df = pa_df.transpose()
pa_df[[0,2]] = pa_df[[0,2]].astype(int)


pa_df.rename(columns={0: 'Number of Unique Items',
                      1: 'Average Purchase Price',
                      2: 'Total Number of Purchases',
                      3: 'Total Revenue'},inplace=True)
pa_df = pa_df.style.format({'Average Purchase Price': '$ {:,.2f}','Total Revenue': '$ {:,.2f}'})
pa_df

,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,63,$ 2.92,78,$ 228.10


# Gender Demographics

In [100]:
gender_counts = df['Gender'].value_counts()

# Percentage and count of male players
male_count = gender_counts['Male']
male_percent = male_count/player_count*100

# Percentage and count of female players
female_count = gender_counts['Female']
female_percent = female_count/player_count*100

# Percentage and count of other/non-disclosed
other_count = gender_counts['Other / Non-Disclosed']
other_percent = other_count/player_count*100

# Compiling analysis into a single table
gender_df = pd.DataFrame([[male_count, male_percent], [female_count, female_percent], [other_count, other_percent]],index=['Male','Female','Other/Non-Disclosed'],columns=['Count','Percent'])
gender_df

,Count,Percent
Male,64,86.486486
Female,13,17.567568
Other/Non-Disclosed,1,1.351351


# Purchase Analysis (Gender)

In [110]:
male_only_df = df[df['Gender']=='Male']
female_only_df = df[df['Gender']=='Female']
other_only_df = df[df['Gender']=='Other / Non-Disclosed']

# Total Purchases
male_purchase_count = len(male_only_df['Price'])
female_purchase_count = len(female_only_df['Price'])
other_purchase_count = len(other_only_df['Price'])

# Average Purchase Price
male_avg_price = male_only_df['Price'].mean()
female_avg_price = female_only_df['Price'].mean()
other_avg_price = other_only_df['Price'].mean()

# Total Purchase Value
male_total_purchase = male_only_df['Price'].sum()
female_total_purchase = female_only_df['Price'].sum()
other_total_purchase = other_only_df['Price'].sum()

# Normalized Totals
# I don't know what this means

# Compiling analysis into a single table
pa_gender_df = pd.DataFrame([[male_purchase_count, male_avg_price, male_total_purchase],
                             [female_purchase_count, female_avg_price, female_total_purchase],
                             [other_purchase_count, other_avg_price, other_total_purchase]],
                             index=['Male','Female','Other/Non-Disclosed'],
                             columns=['Purchase Count','Average Purchase Price','Total Revenue'])
pa_gender_df = pa_gender_df.style.format({'Average Purchase Price': '$ {:,.2f}','Total Revenue': '$ {:,.2f}'})
pa_gender_df

,Purchase Count,Average Purchase Price,Total Revenue
Male,64,$ 2.88,$ 184.60
Female,13,$ 3.18,$ 41.38
Other/Non-Disclosed,1,$ 2.12,$ 2.12
